##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# コードを TensorFlow 2 に自動的にアップグレードする

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/upgrade">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     	TensorFlow.orgで表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/upgrade.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     	Google Colabで実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/upgrade.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/upgrade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>
</td>
</table>


TensorFlow 2.0には引数の並べ替え、シンボル名の変更、パラメータのデフォルト値の変更など、多くのAPIの変更が含まれています。これらの変更をすべて手動で実行するのは退屈で、エラーが発生しやすくなります。変更を合理化し、可能な限りシームレスに TF 2.0 に移行できるよう、TensorFlow チームはレガシーコードの新しい API への移行を支援する `tf_upgrade_v2` ユーティリティを作成しています。

注意: `tf_upgrade_v2` は TensorFlow 1.13 以降（すべての TF 2.0 ビルドを含む）に自動的にインストールされています。

一般的な使用方法は以下のとおりです。

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">tf_upgrade_v2 \<br>  --intree my_project/ \<br>  --outtree my_project_v2/ \<br>  --reportfile report.txt</pre>

これにより、既存の TensorFlow 1.x Python スクリプトが TensorFlow 2.0 に変換され、アップグレード処理が高速化します。

この変換スクリプトは可能な限り自動化を行いますが、スクリプトでは実行できない構文やスタイルの変更もあります。

## 互換性モジュール

いくつかの API シンボルは単に文字列置換を使用するだけではアップグレードできません。コードを確実に TensorFlow 2.0 に対応させるため、アップグレードスクリプトには `compat.v1` モジュールが含まれています。このモジュールは `tf.foo` のような TF 1.x のシンボルを同等の `tf.compat.v1.foo` 参照に置換します。この互換性モジュールは優れていますが、置換箇所を手作業で見直し、それらを `tf.compat.v1` 名前空間ではなく `tf.*` 名前空間の新しい API に早急に移行することをお勧めします。

TensorFlow 2.x で廃止されているモジュール（`tf.flags` や `tf.contrib` など）があるため、一部の変更は `compat.v1` に切り替えても対応できません。このようなコードをアップグレードするには、追加のライブラリ（[`absl.flags`](https://github.com/abseil/abseil-py) など）を使用するか、[tensorflow/addons](http://www.github.com/tensorflow/addons) にあるパッケージに切り替える必要があるかもしれません。


## 推奨アップグレード手順

このガイドの残りの部分では、アップグレードスクリプトの使用方法を説明します。アップグレードスクリプトは簡単に使用できますが、次の手順の一環として使用することを強く推奨します。

1. **単体テスト**: アップグレード対象のコードにカバレッジ率が適度な単体テストスイートを確実に用意します。このコードは Python で記述されているため、さまざまなミスから保護されることはありません。また、すべての依存物が TensorFlow 2.0 との互換性を確保できるようにアップグレード済みであることを確認してください。

2. **TensorFlow 1.14 のインストール**: TensorFlow を最新の TensorFlow 1.x バージョン（1.14 以上）にアップグレードします。このバージョンには `tf.compat.v2` に最終的な TensorFlow 2.0 API が含まれています。

3. **1.14 でテスト**: この時点で単体テストに合格することを確認します。単体テストはアップグレード中に何度も実行することになるため、安全な状態で開始することが重要です。

4. **アップグレードスクリプトの実行**: テストを含むソースツリー全体で `tf_upgrade_v2` を実行します。これにより、TensorFlow 2.0 で利用できるシンボルのみを使用する形式にコードがアップグレードされます。廃止されたシンボルは `tf.compat.v1` でアクセスできます。このようなシンボルは最終的には手動での対応が必要ですが、すぐに対応する必要はありません。

5. **変換後のテストを TensorFlow 1.14 で実行**: コードは引き続き TensorFlow 1.14 で正常に動作するはずです。もう一度単体テストを実行してください。テストで何らかのエラーが発生する場合は、アップグレードスクリプトにバグがあります。その場合は[お知らせください](https://github.com/tensorflow/tensorflow/issues)。

6. **アップグレードレポートの警告とエラーを確認**: このスクリプトは再確認が必要な変換や、必要な手動対応を説明するレポートファイルを書き出します。たとえば、残っているすべての contrib インスタンスを手動で削除する必要がある場合などです。[RFC で詳細](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md)を確認してください。

7. **TensorFlow 2.0 のインストール**: この時点で TensorFlow 2.0 に切り替えても安全です。

8. **`v1.disable_v2_behavior` でのテスト**: テストの main 関数で `v1.disable_v2_behavior()` を使用してテストをもう一度実行すると、1.14 で実行した場合と同じ結果になるはずです。

9. **V2の動作を有効化**: テストが v2 API を使用して動作するようになったため、v2 の動作をオンにすることを検討し始めることができます。コードの記述方法によっては、若干の変更が必要になる場合があります。詳細については、[移行ガイド](migrate.ipynb)を参照してください。

## アップグレードスクリプトの使用


### セットアップ

始める前に、TensorlFlow 2.0 がインストールされていることを確認してください。

In [ ]:
import tensorflow as tf

print(tf.__version__)

テスト対象のコードがある [tensorflow/models](https://github.com/tensorflow/models) git リポジトリをクローンします。

In [ ]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

### ヘルプを読む

スクリプトは TensorFlow と共にインストールされています。組み込みのヘルプは次のとおりです。

In [ ]:
!tf_upgrade_v2 -h

### TF1 のコード例

単純な TensorFlow 1.0 のスクリプトは次のとおりです。

In [ ]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

TensorFlow 2.0 がインストールされている状態では動作しません。

In [ ]:
!(cd models/samples/cookbook/regression &amp;&amp; python custom_regression.py)

### 単一ファイル

アップグレードスクリプトは単体の Python ファイルに対して実行できます。

In [ ]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

コードの修正策が見つからない場合、スクリプトはエラーを出力します。 

### ディレクトリツリー

この単純な例を含む一般的なプロジェクトでは、複数のファイルが使用されています。通常はパッケージ全体をアップグレードするため、スクリプトをディレクトリツリーに対して実行することもできます。

In [ ]:
# upgrade the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

`dataset.make_one_shot_iterator` 関数に関して警告が 1 つ表示されていることに注意してください。

これで、スクリプトが TensorFlow 2.0 で動作するようになりました。

`tf.compat.v1` モジュールのため、変換後のスクリプトは TensorFlow 1.14 でも実行されることに注意してください。 

In [ ]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## 詳細レポート

このスクリプトは、詳細な変更のリストも報告します。この例では安全でない可能性のある変換が 1 つ検出され、ファイルの先頭で警告が表示されています。 

In [ ]:
!head -n 20 tree_report.txt

再度 `Dataset.make_one_shot_iterator function` に関して警告が 1 つ表示されていることに注意してください。

その他の場合、重要な変更の根拠が出力されます。

In [ ]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [ ]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [ ]:
!cat dropout_report.txt

変更されたファイルの内容は次のとおりです。スクリプトがどのように引数名を追加し、移動および名前変更された引数を処理しているかに注目してください。

In [ ]:
!cat dropout_v2.py

大規模なプロジェクトでは、若干のエラーが発生する可能性があります。たとえば、deeplab モデルを変換します。

In [ ]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

次のような出力ファイルが生成されました。

In [ ]:
!ls deeplab_v2

しかし、エラーが発生していました。レポートは、実行前に修正する必要があるものを正確に把握するのに役立ちます。最初の 3 つのエラーは次のとおりです。

In [ ]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## "Safety" モード

この変換スクリプトには `tensorflow.compat.v1` モジュールを使用するようにインポートを変更するだけの侵襲性の低い `SAFETY` モードもあります。

In [ ]:
!cat dropout.py

In [ ]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [ ]:
!cat dropout_v2_safe.py

ご覧のとおり、このモードはコードをアップグレードしませんが、TensorFlow 1 のコードを TensorFlow 2 で実行できます。

## 警告

- このスクリプトを実行する前に手動でコードの一部をアップデートしないでください。特に、`tf.argmax` や `tf.batch_to_space` などの引数の順序が変更された関数により、既存コードを誤ってマッピングするキーワード引数が不正に追加されてしまいます。

- このスクリプトは `tensorflow` が `import tensorflow as tf` を使用してインポートされていることを前提としています。

- このスクリプトは引数の順序を変更しません。その代わり、キーワード引数を引数の順序が変更された関数に追加します。

- GitHub リポジトリ内の Jupyter ノートブックと Python ファイルをアップグレードするための便利なツールについては、[tf2up.ml](http://tf2up.ml) をチェックしてください。

アップグレードスクリプトのバグの報告または機能リクエストを行うには、[GitHub](https://github.com/tensorflow/tensorflow/issues) で課題を報告してください。また、TensorFlow 2.0 をテストしている場合は結果をお聞かせください！[TF 2.0 テストコミュニティ](https://groups.google.com/a/tensorflow.org/forum/#!forum/testing)に参加し、質問や議論を [testing@tensorflow.org](mailto:testing@tensorflow.org) 宛に投稿してください。